![head.png](https://github.com/iwh-halle/FinancialDataAnalytics/blob/master/figures/head.jpg?raw=1)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

# Homework

You will need a Git/GitHub repository to submit your course deliverables. Consult [**slides.ipynb**](https://github.com/iwh-halle/FinancialDataAnalytics) for help with the tasks below! If you need further assistance, do not hesitate to open a Q&A at https://github.com/cafawo/FinancialDataAnalytics/discussions

### Task: 

The liquidity position of a firm measured in million is a **generalized Wiener process** with a drift of $0.1$ per month and a variance of $\sigma^2 = 0.16$ per month. The initial cash position is $2.0$. Calculate:

1. 	the expected value and standard deviation in one, six and 12 months.
2.	What is the probability of a negative cash position in 6 and 12 months? 

In [19]:
import numpy as np
from scipy import stats

# Define values
initial_cash_position = 2
drift = 0.1
variance = 0.16

def expected_value_gwp(t, initial_cash_pos, drift):
    """
    Calculate the expected value of a Guaranteed Withdrawal Product (GWP) at time t.

    Parameters:
        t (float): Time period.
        initial_cash_pos (float): Initial cash position.
        drift (float): Drift rate.

    Returns:
        float: Expected value of the GWP.
    """
    return initial_cash_pos + drift * t

def standard_deviation_gwp(t, var):
    """
    Calculate the standard deviation of a Guaranteed Withdrawal Product (GWP) at time t.

    Parameters:
        t (float): Time period.
        var (float): Variance.

    Returns:
        float: Standard deviation of the GWP.
    """
    return np.sqrt(var * t)

# Print the values for months 1, 6, 12
for month in [1, 6, 12]:
    print(f"Month {month}:")
    print(f"Expected value: {expected_value_gwp(month, initial_cash_position, drift)}")
    print(f"Standard deviation: {standard_deviation_gwp(month, variance):,.4f} \n")

# Probability P(X<=0) is equal to the Cumulative Distribution Function CDF(0)
# In a GWP, the liquidity position at any time t is normally distributed with mean E[X(t)] and variance Var[X(t)]
for month in [6, 12]:
    mu = expected_value_gwp(month, initial_cash_position, drift)
    sigma = standard_deviation_gwp(month, variance)
    probability = stats.norm.cdf(0, mu, sigma)
    print(f"Probability for negative cash position in month {month}: {probability * 100:,.5f}%")


Month 1:
Expected value: 2.1
Standard deviation: 0.4000 

Month 6:
Expected value: 2.6
Standard deviation: 0.9798 

Month 12:
Expected value: 3.2
Standard deviation: 1.3856 

Probability for negative cash position in month 6: 0.39817%
Probability for negative cash position in month 12: 1.04607%


### Task: 

The cash flow of a [call option](https://en.wikipedia.org/wiki/Call_option) with strike $K$ at maturity $T$ is given by

$$
max(S_T - K, 0) = (S_T - K)^+
$$

where $S_T$ is the price of the underlying at $T$. The price of the option under the [risk-neutral measure](https://en.wikipedia.org/wiki/Risk-neutral_measure) $\mathbb{Q}$ is simply its discounted expected value
$$
\mathbb{E}^\mathbb{Q}[(S_T - K)^+] e^{-rT}.
$$


Calculate the price of the option, using:
1. numerical integration and
2. Monte carlo simulation.

For you calculations, assume that todays price of the underlying is $S_0 = 220$, the strike is $K = 220$, volatility is $\sigma = 0.98$, the risk free rate is $r = 10\%$ (continuous) and maturity is one year. We further assume that the underlying $S$ follows a **Geometric Brownian motion**.

Hint: The terminal stock price $S_T$, under the risk-neutral measure, follows a log-normal distribution with PDF

$$f(x) = \frac{1}{x s \sqrt{2 \pi}} \exp\left( -\frac{(\ln x - \mu)^2}{2 s^2} \right) $$

where $\mu = \ln S_0 + (r-\sigma^2 / 2)T$ and variance $s^2 = \sigma^2 T$.


In [20]:
import numpy as np
from scipy import stats
from scipy import integrate
from scipy import optimize
import matplotlib.pyplot as plt

# Define parameters
s_zero = 220  # Initial stock price
strike_K = 220  # Strike price of the option
volatility_sigma = 0.98  # Volatility of the stock
risk_free_rate = 0.1  # Risk-free interest rate
maturity_T = 1  # Time to maturity of the option

def pdf_terminal_stock_price(x, s_zero, sigma, T, r):
    """
    Probability density function (PDF) of the terminal stock price at maturity.

    Parameters:
        x (float): Terminal stock price.
        s_zero (float): Initial stock price.
        sigma (float): Volatility of the stock.
        T (float): Time to maturity of the option.
        r (float): Risk-free interest rate.

    Returns:
        float: PDF value at x.
    """
    mu = np.log(s_zero) + (r - (sigma ** 2) / 2) * T
    s_square = T * sigma ** 2
    return (1 / (x * np.sqrt(s_square * 2 * np.pi))) * np.exp(-((np.log(x) - mu) ** 2) / (2 * s_square))

def payoff(s_t):
    """
    Calculate the payoff of the option at maturity.

    Parameters:
        s_t (float): Terminal stock price.

    Returns:
        float: Option payoff.
    """
    return np.maximum(s_t - strike_K, 0)

# Calculate the expected payoff of the option using numerical integration
expected_payoff, _ = integrate.quad(lambda x: payoff(x) * pdf_terminal_stock_price(x, s_zero, volatility_sigma, maturity_T, risk_free_rate), 0, np.inf)

# Calculate the price of the option
price = expected_payoff * np.exp(-risk_free_rate * maturity_T)

print(f"The price of the option calculated by numerical integration is {price:,.2f}$")


The price of the option calculated by numerical integration is 89.60$


In [21]:
import numpy as np
from scipy import stats
from scipy import integrate
from scipy import optimize
import matplotlib.pyplot as plt

# Define parameters
s_zero = 220  # Initial stock price
strike_K = 220  # Strike price of the option
volatility_sigma = 0.98  # Volatility of the stock
risk_free_rate = 0.1  # Risk-free interest rate
maturity_T = 1  # Time to maturity of the option

def lognormal(s_zero=s_zero, risk_free_rate=risk_free_rate, volatility_sigma=volatility_sigma, T=maturity_T):
    """
    Generate a random variable following a lognormal distribution.

    Parameters:
        s_zero (float): Initial stock price.
        risk_free_rate (float): Risk-free interest rate.
        volatility_sigma (float): Volatility of the stock.
        T (float): Time to maturity of the option.

    Returns:
        float: Random variable following a lognormal distribution.
    """
    mu = np.log(s_zero) + (risk_free_rate - 0.5 * volatility_sigma ** 2) * T
    s_square = volatility_sigma ** 2 * T
    return np.random.lognormal(mu, np.sqrt(s_square))

def payoff():
    """
    Calculate the payoff of the option at maturity.

    Returns:
        float: Option payoff.
    """
    return np.maximum(lognormal() - strike_K, 0)

def monteCarlo(numIter):
    """
    Perform Monte Carlo simulation to estimate the option price.

    Parameters:
        numIter (int): Number of iterations for Monte Carlo simulation.

    Returns:
        float: Estimated option price.
    """
    r = []
    for i in range(numIter):
        r.append(payoff())
    return np.mean(r)

def price_monte_carlo(numIter):
    """
    Calculate the price of the option using Monte Carlo simulation.

    Parameters:
        numIter (int): Number of iterations for Monte Carlo simulation.

    Returns:
        float: Estimated option price.
    """
    return monteCarlo(numIter) * np.exp(-risk_free_rate * maturity_T)

# Calculate the price of the option using Monte Carlo simulation
price = price_monte_carlo(1000000)

print(f"The price of the option calculated by Monte Carlo Simulation is {price:,.2f}$")


The price of the option calculated by Monte Carlo Simulation is 89.44$
